TIME DEPENDENT $\Omega$ IN PROGRESS

In [1]:

from oqd_compiler_infrastructure import Post, PrettyPrint
from trical.light_matter.interface.Chamber import *
from trical.light_matter.interface.operator import Zero,KetBra,ConstantCoefficient,Annihilation,Creation,Identity

class ConstructHTreeRule(ci.ConversionRule):
    def __init__(self,):
        super().__init__()
        pass

    def map_Chamber(self,model,operands):
        pass

    def map_Chain(self,model,operands):
        N = len(operands["ions"])
        L = len(operands["modes"])

        op = []
        for n,ion in enumerate(operands["ions"]):
            _op = [Identity() if n != k else ion for k in range(N)]
            _op.extend([Identity() for _ in range(L)])
            op.append(_op)

        for l,mode in enumerate(operands["modes"]):
            _op = [Identity() for _ in range(N)]
            _op.extend([Identity() if n != k else mode for k in range(L)])
            op.append(_op)


        return op 

    def map_Laser(self,model,operands):
        pass

    def map_Ion(self,model,operands):
        J = len(model.levels)

        op = Zero()
        for j,level in enumerate(model.levels.values()):
            op += KetBra(i=j,j=j,dims=J) * ConstantCoefficient(value=level.energy)

        return op

    def map_VibrationalMode(self,model,operands):
        return Creation() * Annihilation() * ConstantCoefficient(value=model.eigenfreq)

        pass

ion = Ion(
    mass=171,
    charge=1,
    levels={"0": Level(energy=1), "1": Level(energy=2)},
    transitions={},
)

mode = VibrationalMode(eigenfreq=1, eigenvect=[1, 0, 0])

chain = Chain(ions=[ion], modes=[mode])

lasers = [Laser(wavelength=400e-9, intensity=lambda t: 1.0, detuning=0)]

model = {
    'chain': chain,
    'lasers': lasers,
    'timescale': 1
}

compiler = Post(ConstructHTreeRule())
printer = Post(PrettyPrint())

print(printer(compiler(model)))

dict
  - chain: list
    - 0: list
      - 0: OperatorAdd
        - op1: OperatorAdd
          - op1: Zero
          - op2: OperatorScalarMul
            - op: KetBra
              - i: int(0)
              - j: int(0)
              - dims: int(2)
            - coeff: ConstantCoefficient
              - value: float(1.0)
        - op2: OperatorScalarMul
          - op: KetBra
            - i: int(1)
            - j: int(1)
            - dims: int(2)
          - coeff: ConstantCoefficient
            - value: float(2.0)
      - 1: Identity
    - 1: list
      - 0: Identity
      - 1: OperatorScalarMul
        - op: OperatorMul
          - op1: Creation
          - op2: Annihilation
        - coeff: ConstantCoefficient
          - value: float(1.0)
  - lasers: list
    - 0: Laser
      - wavelength: float(4e-07)
      - intensity: function(<function <lambda> at 0x122185870>)
      - polarization: NoneType(None)
      - wavevector: NoneType(None)
      - detuning: float(0.0)
      - phase

In [1]:
from oqd_compiler_infrastructure import Post, PrettyPrint
from trical.light_matter.interface.Chamber import Chamber, Chain, Laser, VibrationalMode, Ion, Level
from trical.light_matter.compiler.rule.codegen.construct_H_tree import ConstructHTreeRule
from trical.light_matter.utilities import detune
import numpy as np

ion = Ion(
    mass=171,
    charge=1,
    levels={
        "0": Level(energy=1),
        "1": Level(energy=2)
    },
    transitions={}
)

mode = VibrationalMode(eigenfreq=1, eigenvect=[1, 0, 0])

chain = Chain(ions=[ion], modes=[mode])

laser = Laser(
    wavelength=400e-9,
    intensity=lambda t: 1.0,
    wavevector=[1, 0, 0],
    polarization=[1, 0, 0],
)
Delta = 31e3
laser.detune(Delta)

chamber = Chamber(chain=chain, lasers=[laser], B=4e-4, Bhat=[0, 0, 1])

timescale = 1

H_tree = ConstructHTreeRule().map_Laser(chamber, {'lasers': [laser], 'timescale': timescale})

printer = Post(PrettyPrint())
print(printer(H_tree))


TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given

In [1]:
from trical.backend.qutip.passes import time_evolve
from oqd_compiler_infrastructure import Post, PrettyPrint, walk
from trical.misc import constants as cst
from trical.light_matter.compiler.rule.codegen.construct_H_tree import ConstructHTreeRule
from trical.light_matter.compiler.rule.rewrite.laser_parameters import SetLaserWavelengthFromTransition, SetLaserIntensityFromPiTime, ModeCutoff
from trical.light_matter.interface.Species import Ca40
from trical.light_matter.interface.Chamber import *
import numpy as np
from math import sqrt

ion = Ca40([("S1/2", -1/2, "S"), ("D5/2", -1/2, "D")])

chain = Chain(ions=[ion,ion], trap_freqs=[3e6, 3e6, 1e6])

mode = VibrationalMode(eigenfreq=1e6, eigenvect=[1.0, 0.5, 0.2])
chain.modes = [mode]

laser = Laser(
    wavelength=ion.transitions[("S1/2", "D5/2")].level1.energy-ion.transitions[("S1/2", "D5/2")].level2.energy,
    intensity=1,
    phase=0,
    wavevector=np.array([1, 0, 1]) / sqrt(2),
    polarization=np.array([1, 0, -1]) / sqrt(2),
)
laser.detune(mode.eigenfreq - 31e3)  # Detune the laser

chamber = Chamber(chain=chain, lasers=[laser], B=4e-4, Bhat=np.array([0, 0, 1]))

# set_wavelength_rule = SetLaserWavelengthFromTransition(
#     0, ion.transitions[("S1/2", "D5/2")]
# )
# set_wavelength_rule.apply(chamber)

# set_intensity_rule = SetLaserIntensityFromPiTime(
#     0, 2e-6, ion.transitions[("S1/2", "D5/2")]
# )
# set_intensity_rule.apply(chamber)

mode_cutoff_value = 4  
timescale = 1e-6  

construct_h_tree_rule = ConstructHTreeRule(
    ions=chain.ions,
    modes=chain.modes,
    lasers=chamber.lasers,
    timescale=timescale,
    mode_cutoff_value=mode_cutoff_value
)
construct_h_tree_pass = Post(construct_h_tree_rule)
compiled_program = construct_h_tree_pass(chamber)

printer = Post(PrettyPrint())
print(printer(compiled_program))



KeyError: 'chamber'

In [ ]:
from qutip import tensor, basis

duration = 60
times = np.linspace(0, duration,  1000)

psi_ion = basis(2,0)
psi_mot = basis(10,0)

psi_0 = tensor(psi_ion, psi_mot)

op = chain.ion_projector(1, "D")

results = time_evolve(H_tree, psi_0, times, expt_ops=[op], progress_bar= True)
results.quick_plot()

AttributeError: 'Chain' object has no attribute 'ion_projector'